In [1]:

import glob
import json
import csv
import os
import random


In [3]:
! pip install pandas aiofiles

In [7]:
import asyncio
import aiofiles
import pandas as pd


semaphore = asyncio.Semaphore(50)
async def read_json_file(file):
    """Read JSON files and extract required fields asynchronously."""
    async with semaphore:
        async with aiofiles.open(file, "r", encoding="utf-8") as f:
            try:
                data = json.loads(await f.read())
                return data.get("results", [])  # Use get() to avoid KeyError
            except json.JSONDecodeError:
                print(f"❌ Error parsing {file}")
                return []
            
async def write_jsonl(file, data):
    """Write JSON data in JSONL format asynchronously."""
    async with semaphore:
        async with aiofiles.open(file, "a", encoding="utf-8") as f:
            await f.writelines(json.dumps(record) + "\n" for record in data)

DATATYPE="TAVG"
DATASET="GHCND"  
DATE="2024-03-18_2025-03-18"
OUTPUT_FILE_JSON=f"{DATATYPE}_{DATASET}_{DATE}.jsonl"
OUTPUT_FILE_CSV=f"{DATATYPE}_{DATASET}_{DATE}.csv"
# INPUT_DIR=f"./data/{DATE}/{DATASET}/{DATATYPE}/"
INPUT_DIR=f"./data/{DATASET}/{DATATYPE}/{DATE}/"
if os.path.exists(OUTPUT_FILE_JSON):
    os.remove(OUTPUT_FILE_JSON)

async def read_json_files():
 
    all_json_files = glob.glob(f"{INPUT_DIR}*.json")
    
    tasks=[read_json_file(file) for file in all_json_files]
    results = await asyncio.gather(*tasks)
    flattened_results = [item for sublist in results for item in sublist]
   
    return  flattened_results
    
results=await read_json_files()
print(f"Total records: {len(results)}")


Total records: 1989973


In [ ]:
#Write to jsonl asynchronously
await write_jsonl(OUTPUT_FILE_JSON, results)


In [ ]:
#Convert to CSV and check for duplicates
df=pd.read_json(OUTPUT_FILE_JSON, lines=True)
print (df.shape)
#One extra line
df.drop_duplicates(subset=['date','station'], keep='first', inplace=True)
print (df.shape)

(1989973, 5)
(1989972, 5)


In [12]:
df.to_csv(OUTPUT_FILE_CSV, index=False)
